In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

## (1) 시청률 데이터 크롤링
출처 : https://www.nielsenkorea.co.kr/ (닐슨코리아)
- 홈쇼핑 방영시간 및 주변 시간대의 시청률이 홈쇼핑의 판매 실적에 영향을 줄 수 있다고 생각하여 변수를 만들어 보았습니다.

In [2]:
def get_title_rating(addr):
    res = requests.get(addr)
    soup = BeautifulSoup(res.content, 'html.parser') # only top 10
    
    # get title
    title = soup.find_all('td', attrs = {'class': 'tb_txt'})[:10]
    title = [re.sub('	</td>', '', re.sub('<td class="tb_txt">', '', str(t))) for t in title]
    
    # get rating
    percent = soup.find_all('td', attrs = {'align':'center', 'class':'percent'})[:10]
    percent = [re.sub('<td align="center" class="percent">', '', str(p)) for p in percent]
    percent = [re.sub('\r\n                        ', '', str(p)) for p in percent]
    percent = [re.sub('                   </td>', '', str(p)) for p in percent][:10]
    
    return title, percent

각 사이트에서 해당 날짜의 상위 10위 시청률과 상응하는 프로그램명을 가져오는 함수입니다.

In [3]:
def get_all_rating(year, broadType):
    for month in range(1,13):
        for day in range(1,32):
            if (year == 2020) & (month != 6):
                break
                
            if (month == 2) & (day == 29):
                break
            elif (month in [4,6,9,11]) & (day == 31):
                break

            if (month < 10) & (day < 10):
                addr = 'https://www.nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1&sub_menu={}&area=00&begin_date={}0{}0{}'.format(broadType, year, month, day)
            elif (month >= 10) & (day < 10):
                addr = 'https://www.nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1&sub_menu={}&area=00&begin_date={}{}0{}'.format(broadType, year, month, day)
            elif (month < 10) & (day >= 10):
                addr = 'https://www.nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1&sub_menu={}&area=00&begin_date={}0{}{}'.format(broadType, year, month, day)
            else:
                addr = 'https://www.nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1&sub_menu={}&area=00&begin_date={}{}{}'.format(broadType, year, month, day)

            title, rating = get_title_rating(addr)
            rating_table = pd.DataFrame(rating, title).reset_index()
            rating_table.rename(columns = {'index' : 'title', 0 : 'rating'}, inplace = True)
            rating_table["year"], rating_table["month"], rating_table["day"] = year, month, day

            if ((month == 1) & (day == 1)) | ((year == 2020) & (month == 6) & (day == 1)):
                all_rating = rating_table
            else:
                all_rating = pd.concat([all_rating, rating_table]) 
    
    all_rating.reset_index(drop = True, inplace = True)
    return all_rating

- 지상파 : 1_1
- 종편 : 2_1
- 케이블 : 3_1
- 2019년 전체와 2020년 6월의 지상파, 종편, 케이블의 날짜별 상위 10위 시청률과 프로그램명을 가져오는 함수입니다.
- 위의 get_title_rating 함수를 사용합니다.

In [4]:
# 2019
jisang19 = get_all_rating(2019, "1_1")
jong19 = get_all_rating(2019, "2_1")
cable19 = get_all_rating(2019, "3_1")

# 2020
jisang20 = get_all_rating(2020, "1_1")
jong20 = get_all_rating(2020, "2_1")
cable20 = get_all_rating(2020, "3_1")

In [5]:
jisang19.to_csv("../eda/data/rating/jisang19.csv")
jong19.to_csv("../eda/data/rating/jong19.csv")
cable19.to_csv("../eda/data/rating/cable19.csv")

jisang20.to_csv("../eda/data/rating/jisang20.csv")
jong20.to_csv("../eda/data/rating/jong20.csv")
cable20.to_csv("../eda/data/rating/cable20.csv")

In [6]:
jisang19.head(10)

,title,rating,year,month,day
0,일일연속극(비켜라운명아),15.6,2019,1,1
1,KBS9시뉴스,11.8,2019,1,1
2,신년특선영화(관상1부),11.8,2019,1,1
3,KBS뉴스7,10.8,2019,1,1
4,신년특선영화(관상2부),10.3,2019,1,1
5,KBS뉴스(09:30),9.9,2019,1,1
6,삼일운동특별기획나의독립영웅,9.8,2019,1,1
7,아침마당,9.8,2019,1,1
8,6시내고향,9.3,2019,1,1
9,인간극장,9.3,2019,1,1


이런식으로 구성되어있습니다.

## (2) 데이터 가공
- 크롤링해 온 시청률 정보를 가지고 조원들끼리 분배하여 해당 프로그램의 방영시간을 조사해왔습니다.
- 그다음 broad_final에 merge해서 하나의 파일로 만들었습니다.
- 이 부분의 코드는 메인 파일에 있습니다.